# Azure Synapse - Workload Management (WM)

## Explore the SQL Pool

In [1]:
-- in master context
select * from sys.database_service_objectives


: Msg 104385, Level 16, State 1, Line 1
Catalog view 'database_service_objectives' is not supported in this version.

Total execution time: 00:00:00.043

Based on this pool size, check the max # queries in   
[Memory and concurrency limits - Azure Synapse Analytics | Microsoft Docs](https://docs.microsoft.com/en-us/azure/synapse-analytics/sql-data-warehouse/memory-concurrency-limits)

In [13]:
--Create a proc to be used to test these limits
CREATE PROC largeQuery
AS
SELECT dc.[Country], dc.[City], dd.[Calendar Year Label], dst.[Stock Item],
FORMAT(COUNT_BIG(f.[Sale Key]),'N0') AS 'Count of Sales',
FORMAT(SUM(f.[Total Excluding Tax]),'N') as 'Revenue',
FORMAT(SUM(f.[Profit]),'N') as 'Profit'
FROM wwi.fact_sale f
JOIN wwi.dimension_City dc ON f.[City Key]=dc.[City Key]
JOIN wwi.dimension_Date dd ON f.[Invoice Date Key]=dd.[Date]
JOIN wwi.dimension_StockItem dst ON f.[Stock Item Key]=dst.[Stock Item Key]
GROUP BY dc.[City], dc.[Country], dd.[Calendar Year Label], dst.[Stock Item]
ORDER BY SUM(f.[Profit]) DESC

GO 

CREATE PROC smallQuery
AS
Select dc.[Country],dc.[City],Format(SUM(f.[Profit]),'N') as 'Profit' from wwi.fact_sale f
Join wwi.dimension_City dc on f.[City Key]=dc.[City Key]
Group By dc.[City],dc.[Country]
ORDER BY SUM(f.[Profit]) DESC


Commands completed successfully.

Total execution time: 00:00:00.308

Monitor one execution and see what is the resource class value

In [10]:
SELECT r.request_id,r.session_id,s.login_name,r.Status,resource_class,[label],resource_class,importance,group_name,classifier_name,resource_allocation_percentage,total_elapsed_time,Command,result_cache_hit
FROM sys.dm_pdw_exec_requests r cross APPLY (SELECT TOP 1 login_name from sys.dm_pdw_exec_sessions where session_id=r.session_id) s
Where r.session_id<>session_id() and (r.Status='Running' or r.status='Suspended')
ORDER By start_time desc;

(4 rows affected)

Total execution time: 00:00:00.644

request_id,session_id,Status,resource_class,label,resource_class,importance,group_name,classifier_name,resource_allocation_percentage,total_elapsed_time,Command,result_cache_hit
QID73631,SID4916,Running,myBigQueries,NULL,myBigQueries,high,myBigQueries,myBigQueriesClassifier,60.00,176482,"SELECT dc.[Country], dc.[City], dd.[Calendar Year Label], dst.[Stock Item], FORMAT(COUNT_BIG(f.[Sale Key]),'N0') AS 'Count of Sales', FORMAT(SUM(f.[Total Excluding Tax]),'N') as 'Revenue', FORMAT(SUM(f.[Profit]),'N') as 'Profit' FROM wwi.fact_sale f JOIN wwi.dimension_City dc ON f.[City Key]=dc.[City Key] JOIN wwi.dimension_Date dd ON f.[Invoice Date Key]=dd.[Date] JOIN wwi.dimension_StockItem dst ON f.[Stock Item Key]=dst.[Stock Item Key] GROUP BY dc.[City], dc.[Country], dd.[Calendar Year Label], dst.[Stock Item] ORDER BY SUM(f.[Profit]) DESC",-64
QID73630,SID4916,Running,NULL,NULL,NULL,NULL,NULL,NULL,NULL,176498,"SELECT dc.[Country], dc.[City], dd.[Calendar Year Label], dst.[Stock Item], FORMAT(COUNT_BIG(f.[Sale Key]),'N0') AS 'Count of Sales', FORMAT(SUM(f.[Total Excluding Tax]),'N') as 'Revenue', FORMAT(SUM(f.[Profit]),'N') as 'Profit' FROM wwi.fact_sale f JOIN wwi.dimension_City dc ON f.[City Key]=dc.[City Key] JOIN wwi.dimension_Date dd ON f.[Invoice Date Key]=dd.[Date] JOIN wwi.dimension_StockItem dst ON f.[Stock Item Key]=dst.[Stock Item Key] GROUP BY dc.[City], dc.[Country], dd.[Calendar Year Label], dst.[Stock Item] ORDER BY SUM(f.[Profit]) DESC",NULL
QID73629,SID4916,Running,NULL,NULL,NULL,high,NULL,myBigQueriesClassifier,NULL,176545,exec largeQuery,NULL
QID73628,SID4916,Running,NULL,NULL,NULL,NULL,NULL,NULL,NULL,176560,exec largeQuery,NULL


Why do we use resoruce class?

Now use ostress to run multiple copies of smallQuery

'''  
.\\Execute-ostress.ps1 -loginName Alice -serverName \*\*\*\*\*\*\*\*.sql.azuresynapse.net -databaseName sampleDatabase -numberOfConnections 30 -numberOfIterations 8 -storedProcName largeQuery

'''

In [ ]:
select count(*) as 'Request_Count',Status from sys.dm_pdw_exec_requests
where [Status] NOT IN ('Completed','Failed','Cancelled') group by Status  --monitor for waiting requests

(1 row affected)

Total execution time: 00:00:00.164

Request_Count,Status
1,Running


In [ ]:
--monitor a waiting request

DECLARE @request_id varchar(20)='QID80368';

SELECT waits.session_id,
      waits.request_id,  
      requests.command,
      requests.status,
      requests.start_time,  
      waits.type,
      waits.state,
      waits.object_type,
      waits.object_name
FROM   sys.dm_pdw_waits waits
   JOIN  sys.dm_pdw_exec_requests requests
   ON waits.request_id=requests.request_id
WHERE waits.request_id = @request_id
ORDER BY waits.object_name, waits.object_type, waits.state;


## Workload group

In [5]:
CREATE WORKLOAD GROUP largeWorkloadGroup
 WITH
 (   MIN_PERCENTAGE_RESOURCE = 30 
   , CAP_PERCENTAGE_RESOURCE = 100 
   , REQUEST_MIN_RESOURCE_GRANT_PERCENT = 10
   , REQUEST_MAX_RESOURCE_GRANT_PERCENT = 80 
   , IMPORTANCE =  ABOVE_NORMAL 
   , QUERY_EXECUTION_TIMEOUT_SEC = 7200 )

Commands completed successfully.

Total execution time: 00:00:00.117

In [1]:
SELECT * FROM sys.workload_management_workload_groups

(13 rows affected)

Total execution time: 00:00:00.083

group_id,name,importance,min_percentage_resource,cap_percentage_resource,request_min_resource_grant_percent,request_max_resource_grant_percent,query_execution_timeout_sec,query_wait_timeout_sec,create_time,modify_time
1,smallrc,normal,0,100,3.00,3.00,0,0,2021-01-27 14:33:51.550,2021-01-27 14:33:51.550
2,mediumrc,normal,0,100,10.00,10.00,0,0,2021-01-27 14:33:51.557,2021-01-27 14:33:51.557
3,largerc,normal,0,100,22.00,22.00,0,0,2021-01-27 14:33:51.560,2021-01-27 14:33:51.560
4,xlargerc,normal,0,100,70.00,70.00,0,0,2021-01-27 14:33:51.567,2021-01-27 14:33:51.567
5,staticrc10,normal,0,100,0.40,0.40,0,0,2021-01-27 14:33:51.570,2021-01-27 14:33:51.570
6,staticrc20,normal,0,100,0.80,0.80,0,0,2021-01-27 14:33:51.573,2021-01-27 14:33:51.573
7,staticrc30,normal,0,100,1.60,1.60,0,0,2021-01-27 14:33:51.580,2021-01-27 14:33:51.580
8,staticrc40,normal,0,100,3.20,3.20,0,0,2021-01-27 14:33:51.583,2021-01-27 14:33:51.583
9,staticrc50,normal,0,100,6.40,6.40,0,0,2021-01-27 14:33:51.590,2021-01-27 14:33:51.590
10,staticrc60,normal,0,100,12.80,12.80,0,0,2021-01-27 14:33:51.593,2021-01-27 14:33:51.593


## Calssifier

In [4]:
--for a user 
CREATE WORKLOAD CLASSIFIER myBigQueriesClassifier
  WITH (WORKLOAD_GROUP = 'largeWorkloadGroup'
       ,MEMBERNAME = 'Alice')
     

Commands completed successfully.

Total execution time: 00:00:00.106

In [ ]:
--dynamic using label
CREATE WORKLOAD CLASSIFIER myBigQueriesClassifier2
  WITH (WORKLOAD_GROUP = 'largeWorkloadGroup'
       ,MEMBERNAME = 'public'
      ,WLM_LABEL='big_label' )

Commands completed successfully.

Total execution time: 00:00:00.072

In [ ]:
--dynamic using session context 
CREATE WORKLOAD CLASSIFIER myBigQueriesClassifier3
  WITH (WORKLOAD_GROUP = 'largeWorkloadGroup'
       ,MEMBERNAME = 'public'
       ,WLM_CONTEXT='big_context' )

Commands completed successfully.

Total execution time: 00:00:00.115

In [3]:
SELECT * FROM sys.workload_management_workload_classifiers c join sys.workload_management_workload_classifier_details cd on c.classifier_id=cd.classifier_id
where c.name='myBigQueriesClassifier' or c.name='myBigQueriesClassifier2' or c.name='myBigQueriesClassifier3'


(5 rows affected)

Total execution time: 00:00:00.076

classifier_id,name,group_name,importance,create_time,modify_time,is_enabled,classifier_id,classifier_type,classifier_value
3019,myBigQueriesClassifier,largeWorkloadGroup,NULL,2021-03-15 01:40:20.003,2021-03-15 01:40:20.003,1,3019,membername,Alice
3020,myBigQueriesClassifier2,largeWorkloadGroup,NULL,2021-03-15 01:40:21.213,2021-03-15 01:40:21.213,1,3020,membername,public
3020,myBigQueriesClassifier2,largeWorkloadGroup,NULL,2021-03-15 01:40:21.213,2021-03-15 01:40:21.213,1,3020,wlm_label,big_label
3021,myBigQueriesClassifier3,largeWorkloadGroup,NULL,2021-03-15 01:40:23.017,2021-03-15 01:40:23.017,1,3021,membername,public
3021,myBigQueriesClassifier3,largeWorkloadGroup,NULL,2021-03-15 01:40:23.017,2021-03-15 01:40:23.017,1,3021,wlm_context,big_context


## Use labels & session context

In [ ]:
--session context 
EXEC sys.sp_set_session_context @key = 'wlm_context', @value = 'big_context'

--label 
SELECT dc.[Country], dc.[City], dd.[Calendar Year Label], dst.[Stock Item],
FORMAT(COUNT_BIG(f.[Sale Key]),'N0') AS 'Count of Sales',
FORMAT(SUM(f.[Total Excluding Tax]),'N') as 'Revenue',
FORMAT(SUM(f.[Profit]),'N') as 'Profit'
FROM wwi.fact_sale f
JOIN wwi.dimension_City dc ON f.[City Key]=dc.[City Key]
JOIN wwi.dimension_Date dd ON f.[Invoice Date Key]=dd.[Date]
JOIN wwi.dimension_StockItem dst ON f.[Stock Item Key]=dst.[Stock Item Key]
GROUP BY dc.[City], dc.[Country], dd.[Calendar Year Label], dst.[Stock Item]
ORDER BY SUM(f.[Profit]) DESC  OPTION (LABEL = 'big_label')

## Clear resources

In [3]:
--clear 
DROP WORKLOAD CLASSIFIER myBigQueriesClassifier;
GO
DROP WORKLOAD CLASSIFIER myBigQueriesClassifier2;
GO
DROP WORKLOAD CLASSIFIER myBigQueriesClassifier3;
GO
DROP WORKLOAD GROUP largeWorkloadGroup

: Msg 105585, Level 16, State 1, Line 1
DROP WORKLOAD CLASSIFIER statement failed since the classifier does not exist

: Msg 105585, Level 16, State 1, Line 4
DROP WORKLOAD CLASSIFIER statement failed since the classifier does not exist

: Msg 105585, Level 16, State 1, Line 6
DROP WORKLOAD CLASSIFIER statement failed since the classifier does not exist

Commands completed successfully.

Total execution time: 00:00:00.467